**Meysam Amirsardari**
SID: 98106218

Loading Datasets from Kaggle: 

In [ ]:
! pip install kaggle
#! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d aashwingupta3012/human-faces

#https://www.kaggle.com/datasets/niten19/face-shape-dataset
#https://www.kaggle.com/datasets/greatgamedota/ffhq-face-data-set

In [ ]:
! unzip "/content/human-faces.zip"

Imports:

In [ ]:
import os
from time import time
import errno
import shutil
import cv2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import fetch_lfw_people
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, log_loss, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import GridSearchCV
import seaborn as sns
import seaborn as sns; sns.set()
from skimage import data, color, feature
import skimage.data
from sklearn.feature_extraction.image import PatchExtractor
from skimage import data, transform
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB

Faces:

In [ ]:
people = fetch_lfw_people()
faceImages = people.images

In [ ]:
x = []
y = []
descriptor = cv2.HOGDescriptor()

for i in range(len(faceImages)):
    image = cv2.resize(faceImages[i], (128, 128))
    image = feature.hog(image)
    image = image.transpose()
    x.append(image)
    y.append(1)

In [ ]:
def extract_patches(img, N, scale=1.0, size=faceImages[0].shape):
    extracted_patch_size = tuple((scale * np.array(size)).astype(int))
    extractor = PatchExtractor(patch_size=extracted_patch_size,
                               max_patches=N, random_state=0)
    patches = extractor.transform(img[np.newaxis])
    if scale != 1:
        patches = np.array([transform.resize(patch, size)
                            for patch in patches])
    return patches

imgs_to_use = ['camera', 'text', 'coins', 'moon',
               'page', 'clock', 'immunohistochemistry',
               'chelsea', 'coffee', 'hubble_deep_field']
images = [color.rgb2gray(getattr(data, name)())
          for name in imgs_to_use]

nonfaces = np.vstack([extract_patches(im, 1000, scale)
                              for im in images for scale in [0.5, 1.0, 2.0]])
nonfaces = nonfaces[0:10000]

for i in range(nonfaces.shape[0]):
    image = cv2.resize(nonfaces[i, :, :], (128, 128))
    image = feature.hog(image)
    image = image.transpose()
    x.append(image)
    y.append(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: The behavior of rgb2gray will change in scikit-image 0.19. Currently, rgb2gray allows 2D grayscale image to be passed as inputs and leaves them unmodified as outputs. Starting from version 0.19, 2D arrays will be treated as 1D images with 3 channels.
  from ipykernel import kernelapp as app


**More data:**

In [ ]:
base = '/content/afhq/train/wild'
names = os.listdir(base)

for i,name in enumerate(names):
    img = cv2.imread(os.path.join(base, name), 0)
    image = cv2.resize(img, (64, 64))
    image = feature.hog(image)
    image = image.transpose()
    x.append(image)
    y.append(0)

**DON'T RUN THIS!!!!**

In [ ]:
winSize = (64,64)
blockSize = (16,16)
blockStride = (8,8)
cellSize = (8,8)
nbins = 9
derivAperture = 1
winSigma = 4
histogramNormType = 0
L2HysThreshold = 2.01e-01
gammaCorrection = 0
nlevels = 64
hog = cv2.HOGDescriptor(winSize, blockSize, blockStride, cellSize, nbins,derivAperture, winSigma,
                        histogramNormType, L2HysThreshold, gammaCorrection, nlevels)
winStride = (8,8)
padding = (8,8)
locations = ((10,20),)
out = hog.compute(image, winStride, padding, locations)

Training:

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.05, shuffle=True)

cross_val_score(GaussianNB(), X_train, y_train)

array([0.97100793, 0.97258722, 0.96896239, 0.97077481, 0.96964205])

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.model_selection  import GridSearchCV
grid = GridSearchCV(LinearSVC(), {'C': [1.0, 2.0, 4.0, 8.0]})
grid.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1198: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  sample_weight,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1198: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  sample_weight,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1198: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  sample_weight,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1198: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  sample_weight,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1198: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  sample_weight,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1198: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  sample_

GridSearchCV(estimator=LinearSVC(), param_grid={'C': [1.0, 2.0, 4.0, 8.0]})

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

model = grid.best_estimator_
model.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1198: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  sample_weight,


LinearSVC()

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Acc = %.2lf" % (accuracy*100))

Acc = 99.31


In [ ]:
import joblib
joblib.dump(model, 'SVMmodel.sav')

['SVMmodel.sav']

Reloading trained model:

In [ ]:
import pickle

model = pickle.load(open('SVMmodel.sav', 'rb'))

**Test1:**

In [ ]:
import imutils
from google.colab.patches import cv2_imshow

def sliding_window(image, stepSize, windowSize):
  for y in range(0, image.shape[0], stepSize):
    for x in range(0, image.shape[1], stepSize):
      yield (x, y, image[y:y + windowSize[1], x:x + windowSize[0]])

def pyramid(image, scale=1.4, minSize=(32, 32)):
    yield image 

    while True:
        w = int(image.shape[1] / scale)     
        image = imutils.resize(image, width=w)

        if image.shape[0] < minSize[1] or image.shape[1] > minSize[0]:
            break 
        yield image

test_image = skimage.data.astronaut()
test_image = skimage.color.rgb2gray(test_image)
test_image = skimage.transform.rescale(test_image, 0.5)
image = test_image[:160, 40:180]

bboxes = []
(winW, winH) = (128, 128)

for resized in pyramid(image, scale=1.4):
  for (x, y, window) in sliding_window(resized, stepSize=32, windowSize=(winW, winH)):
    if window.shape[0] != winH or window.shape[1] != winW: continue
    fd = feature.hog(window)
    pred = model.predict([fd])
    
    if pred==1:
      bboxes.append((x, y, winW, winH))
      
    clone = resized.copy()
    cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
    cv2_imshow(clone)
    cv2.waitKey(1)

X = []
Y = []
Z = []

for(x, y, w, h) in bboxes:
  image = cv2.rectangle(image, (x, y), 
                    (x + w, y + h), 
                    (255, 0, 0), 2)  
  X.append(x+w/2)
  Y.append(y+h/2)
  Z.append(1)  

cv2_imshow(image)

Density map:

In [ ]:
import pylab as plt
import numpy as np

plt.pcolormesh(X,Y,Z)
plt.show()

**Test2 (Haar cascades):**

In [ ]:
def non_max_suppression_slow(boxes, overlapThresh):
	if len(boxes) == 0:
		return []

	pick = []
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]

	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)
 
  while len(idxs) > 0:
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		suppress = [last]
    for pos in xrange(0, last):
			j = idxs[pos]

			xx1 = max(x1[i], x1[j])
			yy1 = max(y1[i], y1[j])
			xx2 = min(x2[i], x2[j])
			yy2 = min(y2[i], y2[j])

			w = max(0, xx2 - xx1 + 1)
			h = max(0, yy2 - yy1 + 1)

			overlap = float(w * h) / area[j]

			if overlap > overlapThresh:
				suppress.append(pos)

		idxs = np.delete(idxs, suppress)
	return boxes[pick]

In [ ]:
from google.colab.patches import cv2_imshow

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

image = cv2.imread('/content/img3.jpg')

height, width = image.shape[:2]

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + '/haarcascade_frontalface_default.xml')

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

min_size = int(max(0.001*width, 0.001*height))
BBoxes = face_cascade.detectMultiScale(gray,
                                      scaleFactor=1.1,
                                      minNeighbors=2,
                                      minSize=(min_size, min_size))

X = []
Y = []
Z = []

#BBoxes = non_max_suppression_slow(boundingBoxes, 0.3)

for (x, y, w, h) in BBoxes:
    cv2.rectangle(image, (x, y), 
                  (x + w, y + h), 
                  (0, 0, 255), 2)
    X.append(x+w/2)
    Y.append(y+h/2)
    Z.append(1)
  
# cv2_imshow(image)
# cv2.waitKey(0)   
# cv2.destroyAllWindows()

In [ ]:
test_image = skimage.data.astronaut()
test_image = skimage.color.rgb2gray(test_image)
test_image = skimage.transform.rescale(test_image, 0.5)
image = test_image[:160, 40:180]

height, width = image.shape[:2]
min_size = int(max(0.1*width, 0.1*height))
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + '/haarcascade_frontalface_default.xml')


In [ ]:
import pylab as plt
import numpy as np

plt.pcolormesh(X,Y,Z)
plt.show()